### 1. Import Libraries

In [7]:
import yfinance as yf
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from itertools import combinations

### 2. Data Obtention

In [8]:
# Lista de empresas a evaluar
tickers_list = ['CLF', 'NUE', 'ZEUS', 'STLD', 'RS', 'MTUS', 'FRD', 'X', 'MSB', 'CMC', 'ACNT', 'RDUS']

# Descargar datos históricos
tickers = yf.Tickers(" ".join(tickers_list))
hist = tickers.history(start='2021-01-01',end='2024-12-31')
adj_close = hist['Close'].dropna(axis=1)  # Eliminar valores NaN
print(adj_close)

[*********************100%***********************]  12 of 12 completed

Ticker       ACNT        CLF        CMC        FRD        MSB   MTUS  \
Date                                                                   
2021-01-04   8.00  14.420000  19.221788   6.512207  16.668919   4.84   
2021-01-05   8.01  15.700000  19.898546   6.676219  17.464701   5.17   
2021-01-06   8.25  17.420000  22.079210   6.801640  17.495071   5.56   
2021-01-07   8.73  17.670000  21.806627   6.927416  17.173117   5.64   
2021-01-08   8.95  18.040001  21.205065   6.937091  16.772188   5.69   
...           ...        ...        ...        ...        ...    ...   
2024-12-23  11.01   9.370000  49.606380  15.884322  22.506735  14.12   
2024-12-24  11.01   9.460000  50.323872  15.804552  23.372063  14.27   
2024-12-26  11.36   9.350000  50.891884  15.814523  23.569851  14.31   
2024-12-27  11.28   9.240000  49.935230  15.525354  22.877588  14.09   
2024-12-30  11.25   9.300000  49.048336  15.016817  23.174273  13.73   

Ticker             NUE       RDUS          RS        STLD      

### 3. Portfolios Analysis

In [9]:
# Función para calcular métricas del portafolio
def calcular_metricas(adj_close_values):
    R = np.log(adj_close_values[1:] / adj_close_values[:-1])  # Retornos logarítmicos
    RE = np.mean(R, axis=0) * 252  # Retorno esperado anualizado
    RI = np.std(R, axis=0) * np.sqrt(252)  # Riesgo anualizado
    S = np.cov(R, rowvar=False)  # Matriz de covarianza
    corr = np.corrcoef(R, rowvar=False)  # Matriz de correlación
    return RE, RI, S, corr

# Obtener métricas generales
adj_close_values = adj_close.values
RE, RI, S, correlation_matrix = calcular_metricas(adj_close_values)

# Evaluar todas las combinaciones posibles de 4 activos
mejores_portafolios = []
for subset in combinations(adj_close.columns, 4):
    indices = [adj_close.columns.get_loc(ticker) for ticker in subset]
    corr_submatrix = correlation_matrix[np.ix_(indices, indices)]

    # Filtrar combinaciones con correlación > 0.5
    if np.any(np.abs(np.triu(corr_submatrix, k=1)) > 0.5):
        continue

    # Extraer datos de la combinación aceptable
    RE_sub = RE[indices]
    S_sub = S[np.ix_(indices, indices)]
    weights = np.ones(4) / 4  # Pesos iniciales iguales

    # Definir restricciones y límites
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = [(0, None)] * 4

    # Optimizar el portafolio
    res = minimize(lambda w: w @ S_sub @ w.T, x0=weights, method='SLSQP', bounds=bounds, constraints=constraints, options={'disp': False, 'maxiter': 1000, 'ftol': 1e-12})

    if res.success:
        # Calcular métricas del portafolio optimizado
        ReP = res.x @ RE_sub.T
        varP = res.x @ S_sub @ res.x.T
        RiP = np.sqrt(varP)
        SharpeP = ReP / RiP

        # Guardar resultados
        mejores_portafolios.append({
            "Activos": subset,
            "ReP (%)": round(ReP * 100, 4),
            "RiP (%)": round(RiP * 100, 4),
            "SharpeP": round(SharpeP, 4)
        })

# Ordenar por el mejor Sharpe Ratio
mejores_portafolios = sorted(mejores_portafolios, key=lambda x: x["SharpeP"], reverse=True)

# Convertir a DataFrame y mostrar los resultados
df_resultados = pd.DataFrame(mejores_portafolios)
print(df_resultados)

# Guardar en un archivo Excel
df_resultados.to_excel("mejores_portafolios.xlsx", index=False)
print("\nResultados guardados en 'mejores_portafolios.xlsx'")

                   Activos  ReP (%)  RiP (%)  SharpeP
0     (ACNT, FRD, MSB, RS)  17.3895   1.5806  11.0017
1   (ACNT, FRD, MSB, STLD)  18.0853   1.8416   9.8207
2    (ACNT, CMC, FRD, MSB)  16.3599   1.7800   9.1911
3    (ACNT, FRD, MSB, NUE)  14.9913   1.8251   8.2140
4   (ACNT, FRD, MSB, MTUS)  13.4665   2.0004   6.7318
5   (ACNT, FRD, MSB, ZEUS)  12.8674   2.0018   6.4279
6      (ACNT, FRD, MSB, X)  12.1779   1.9690   6.1848
7    (ACNT, CLF, FRD, MSB)   8.2876   1.9945   4.1553
8      (FRD, MSB, RDUS, X)   7.4109   2.1462   3.4530
9     (ACNT, FRD, RDUS, X)   6.9768   2.1134   3.3013
10  (ACNT, FRD, MSB, RDUS)   5.9856   1.9616   3.0514
11    (ACNT, MSB, RDUS, X)   4.8278   2.0367   2.3705

Resultados guardados en 'mejores_portafolios.xlsx'
